In [ ]:
# === Celda 1: Crear pipeline con StandardScaler ya ajustado y guardar ===
import os
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Rutas (ajusta si usas otras)
PATH_DF = '/content/df_limpio_modelo.csv'              # CSV usado en entrenamiento (contiene 'target')
MODEL_FILE = '/content/mejor_modelo_AdaBoost.pkl'      # tu modelo AdaBoost entrenado
OUT_PIPE = '/content/modelo_final_pipeline.pkl'       # pipeline resultante (salida)

# 1) Chequeos iniciales
for p in [PATH_DF, MODEL_FILE]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"No se encontró el archivo requerido: {p}. Súbelo a /content y vuelve a ejecutar.")

# 2) Cargar datos de entrenamiento y obtener X (sin target)
df = pd.read_csv(PATH_DF)
if 'target' not in df.columns:
    raise KeyError("El CSV de entrenamiento debe contener la columna 'target'. Cambia el nombre o actualiza el CSV.")
X = df.drop(columns=['target'])
feature_names = X.columns.tolist()
print("✅ Columnas (features) detectadas:", feature_names)

# 3) Cargar el estimador AdaBoost (o pipeline que contiene solo el modelo)
modelo = joblib.load(MODEL_FILE)
print("✅ Modelo cargado:", type(modelo).__name__)

# 4) Ajustar el scaler en los datos originales (sin modificar el modelo)
scaler = StandardScaler()
scaler.fit(X)
print("✅ StandardScaler ajustado con los datos de entrenamiento.")

# 5) Crear pipeline final (scaler ya ajustado + modelo)
pipeline_completo = Pipeline([
    ('scaler', scaler),
    ('model', modelo)
])

# 6) Guardar pipeline completo
joblib.dump(pipeline_completo, OUT_PIPE)
print(f"✅ Pipeline completo guardado en: {OUT_PIPE}")


✅ Columnas (features) detectadas: ['age', 'sex', 'thalach', 'exang', 'oldpeak', 'ca', 'risk_index', 'age_thalach_interaction', 'oldpeak_ca_ratio', 'cp_1', 'cp_2', 'cp_3', 'restecg_1', 'restecg_2', 'thal_1', 'thal_2', 'thal_3', 'age_group_middle', 'age_group_senior']
✅ Modelo cargado: AdaBoostClassifier
✅ StandardScaler ajustado con los datos de entrenamiento.
✅ Pipeline completo guardado en: /content/modelo_final_pipeline.pkl


In [ ]:
# === Celda 2: Cargar pipeline y predecir sobre nuevos_datos.csv (verifica columnas y orden) ===
import os
import pandas as pd
import joblib
from pathlib import Path

PIPE_PATH = '/content/modelo_final_pipeline.pkl'  # archivo creado en la Celda 1
NUEVOS_CSV = '/content/nuevos_datos.csv'          # CSV con nuevos ejemplos a predecir
DF_ENTRENAMIENTO = '/content/df_limpio_modelo.csv' # para recuperar el orden de columnas usadas

# Comprobaciones
if not os.path.exists(PIPE_PATH):
    raise FileNotFoundError(f"No se encontró el pipeline en: {PIPE_PATH}. Ejecuta la celda 1 primero.")
if not os.path.exists(NUEVOS_CSV):
    raise FileNotFoundError(f"No se encontró {NUEVOS_CSV}. Sube tus datos nuevos a /content/nuevos_datos.csv")

# Cargar pipeline
pipe = joblib.load(PIPE_PATH)
print("✅ Pipeline cargado:", PIPE_PATH)

# Recuperar columnas (orden correcto) desde el CSV de entrenamiento
if not os.path.exists(DF_ENTRENAMIENTO):
    raise FileNotFoundError(f"No se encontró {DF_ENTRENAMIENTO}. Necesitamos el CSV de entrenamiento para conocer el orden de columnas.")
df_train = pd.read_csv(DF_ENTRENAMIENTO)
if 'target' not in df_train.columns:
    raise KeyError("df_limpio_modelo.csv debe contener la columna 'target'.")
expected_features = df_train.drop(columns=['target']).columns.tolist()
print("✅ Columnas esperadas (en orden):", expected_features)

# Cargar nuevos datos
nuevos = pd.read_csv(NUEVOS_CSV)
print("Nuevos datos leídos. Shape:", nuevos.shape)
print("Columnas en nuevos_datos.csv:", nuevos.columns.tolist())

# Verificar que todos los features esperados estén presentes
missing = [c for c in expected_features if c not in nuevos.columns]
if missing:
    raise KeyError(f"Faltan columnas en nuevos_datos.csv que el modelo espera: {missing}")

# Reordenar y seleccionar solo las columnas esperadas (esto evita problemas de orden)
nuevos_alineados = nuevos[expected_features].copy()
print("✅ Nuevos datos alineados con el orden de entrenamiento. Shape:", nuevos_alineados.shape)

# Realizar predicciones
try:
    preds = pipe.predict(nuevos_alineados)
    # Si el pipeline soporta predict_proba, también mostrar probabilidades (opc.)
    probs = None
    if hasattr(pipe, "predict_proba"):
        try:
            probs = pipe.predict_proba(nuevos_alineados)
        except Exception:
            probs = None

    print("✅ Predicciones realizadas. Ejemplo (primeras 10):", preds[:10])
    if probs is not None:
        print("✅ Ejemplo de probabilidades (primeras 3 filas):")
        print(probs[:3])

    # Guardar resultados junto a los datos de entrada
    resultados = nuevos_alineados.copy()
    resultados['prediction'] = preds
    # si probs existe y tiene 2 columnas (binario), añadir probabilidad positiva
    if probs is not None and probs.shape[1] == 2:
        resultados['probabilidad_positiva'] = probs[:, 1]

    out_file = '/content/predicciones_resultado.csv'
    resultados.to_csv(out_file, index=False)
    print(f"✅ Resultados guardados en: {out_file}")

except Exception as e:
    raise RuntimeError(f"Error durante la predicción: {e}")


✅ Pipeline cargado: /content/modelo_final_pipeline.pkl
✅ Columnas esperadas (en orden): ['age', 'sex', 'thalach', 'exang', 'oldpeak', 'ca', 'risk_index', 'age_thalach_interaction', 'oldpeak_ca_ratio', 'cp_1', 'cp_2', 'cp_3', 'restecg_1', 'restecg_2', 'thal_1', 'thal_2', 'thal_3', 'age_group_middle', 'age_group_senior']
Nuevos datos leídos. Shape: (10, 20)
Columnas en nuevos_datos.csv: ['age', 'sex', 'thalach', 'exang', 'oldpeak', 'ca', 'target', 'risk_index', 'age_thalach_interaction', 'oldpeak_ca_ratio', 'cp_1', 'cp_2', 'cp_3', 'restecg_1', 'restecg_2', 'thal_1', 'thal_2', 'thal_3', 'age_group_middle', 'age_group_senior']
✅ Nuevos datos alineados con el orden de entrenamiento. Shape: (10, 19)
✅ Predicciones realizadas. Ejemplo (primeras 10): [0 1 0 1 1 0 1 0 1 1]
✅ Ejemplo de probabilidades (primeras 3 filas):
[[0.86772889 0.13227111]
 [0.18948607 0.81051393]
 [0.86772889 0.13227111]]
✅ Resultados guardados en: /content/predicciones_resultado.csv


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


In [ ]:
!pip install streamlit pyngrok


In [ ]:
%%writefile /content/app.py
import streamlit as st
import pandas as pd
import joblib
import os
import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Config
MODEL_PATH = '/content/modelo_final_pipeline.pkl'
DF_TRAIN_PATH = '/content/df_limpio_modelo.csv'
PREDICTIONS_OUT = '/content/predicciones_resultado.csv'
VALIDATION_EVIDENCE = '/content/evidencia_validacion_predicciones.csv'
LOGFILE = '/content/prediction_log.csv'

# Configuración de página con tema personalizado
st.set_page_config(
    page_title="Predicción AdaBoost",
    layout="wide",
    initial_sidebar_state="expanded"
)

# CSS personalizado para diseño moderno
st.markdown("""
<style>
/* Fondo general */
.stApp {
    background-color: #f9fafb;
}

/* Contenedor principal */
.main .block-container {
    background: white;
    border-radius: 18px;
    padding: 2rem;
    box-shadow: 0 4px 20px rgba(0, 0, 0, 0.05);
}

/* Título principal */
.main-title {
    color: #1f2937;
    font-size: 2.5rem;
    font-weight: 800;
    text-align: center;
    margin-bottom: 0.5rem;
}

/* Subtítulo */
.subtitle {
    text-align: center;
    font-size: 1.1rem;
    color: #6b7280;
    margin-bottom: 2rem;
}

/* Cards */
.card {
    background: #ffffff;
    border-radius: 12px;
    padding: 1.5rem;
    margin: 1rem 0;
    border: 1px solid #e5e7eb;
    transition: all 0.3s ease;
}
.card:hover {
    box-shadow: 0 4px 12px rgba(0,0,0,0.08);
}

/* Métricas */
.metric-container {
    background: #4f46e5;
    border-radius: 10px;
    padding: 1rem;
    color: white;
    text-align: center;
    margin: 0.5rem;
}
.metric-value {
    font-size: 2rem;
    font-weight: bold;
}
.metric-label {
    font-size: 0.9rem;
    text-transform: uppercase;
    letter-spacing: 1px;
    opacity: 0.9;
}

/* Botones */
.stButton>button {
    background: #4f46e5;
    color: white;
    border: none;
    border-radius: 8px;
    padding: 0.6rem 1.5rem;
    font-weight: 600;
    transition: all 0.3s ease;
}
.stButton>button:hover {
    background: #4338ca;
    transform: translateY(-2px);
}

/* Inputs */
.stNumberInput>div>div>input {
    border-radius: 8px;
    border: 1px solid #d1d5db;
}
.stNumberInput>div>div>input:focus {
    border-color: #4f46e5;
    box-shadow: 0 0 0 3px rgba(79, 70, 229, 0.2);
}

/* Tabs */
.stTabs [data-baseweb="tab-list"] {
    background-color: #f3f4f6;
    border-radius: 8px;
    padding: 0.4rem;
    gap: 6px;
}
.stTabs [data-baseweb="tab"] {
    border-radius: 6px;
    padding: 0.4rem 1rem;
    font-weight: 600;
    color: #374151;
}
.stTabs [aria-selected="true"] {
    background-color: #4f46e5 !important;
    color: white !important;
}

/* File uploader */
.stFileUploader>div>div {
    border-radius: 10px;
    border: 2px dashed #a5b4fc;
    background: #f8fafc;
}

/* Expander */
.streamlit-expanderHeader {
    background: #4f46e5;
    color: white;
    border-radius: 8px;
    font-weight: 600;
}

/* Footer */
.footer {
    text-align: center;
    color: #6b7280;
    font-size: 0.9rem;
    padding: 1.5rem 0;
}
</style>
""", unsafe_allow_html=True)


# Helper: append log
def append_log(input_df, prediction, probs=None):
    df = input_df.copy()
    df['prediction'] = prediction
    if probs is not None and getattr(probs, 'shape', None) and probs.shape[1] == 2:
        df['prob_pos'] = probs[:, 1]
    df['timestamp'] = datetime.datetime.utcnow().isoformat()
    header = not os.path.exists(LOGFILE)
    df.to_csv(LOGFILE, mode='a', header=header, index=False)
    return LOGFILE

# Header con diseño moderno
st.markdown('<h1 class="main-title">🔮 Predicción de datos futuros</h1>', unsafe_allow_html=True)
st.markdown('<p style="text-align: center; font-size: 1.2rem; color: #666; margin-bottom: 2rem;">Sistema de predicción avanzado con AdaBoost</p>', unsafe_allow_html=True)

# Load pipeline con indicador visual mejorado
pipe = None
with st.spinner('🔄 Cargando modelo...'):
    if not os.path.exists(MODEL_PATH):
        st.error(f"❌ Modelo no encontrado en: {MODEL_PATH}. Sube 'modelo_final_pipeline.pkl' a /content")
    else:
        try:
            pipe = joblib.load(MODEL_PATH)
            st.success("✅ Pipeline cargado correctamente")
        except Exception as e:
            st.error(f"❌ Error cargando el modelo: {e}")
            pipe = None

# Infer feature names from training CSV if available
def get_feature_names():
    if os.path.exists(DF_TRAIN_PATH):
        try:
            df_train = pd.read_csv(DF_TRAIN_PATH)
            if 'target' in df_train.columns:
                return df_train.drop(columns=['target']).columns.tolist()
            else:
                return df_train.columns.tolist()
        except Exception:
            return None
    return None

feature_names = get_feature_names() or ['col1', 'col2', 'col3', 'col4']

# Tabs principales para mejor organización
tab1, tab2, tab3 = st.tabs(["🔢 Predicción Manual", "📁 Predicción por Lote", "🧪 Validación"])

# TAB 1: Entrada manual
with tab1:
    if pipe is not None:
        st.markdown('<div class="card">', unsafe_allow_html=True)
        st.subheader("📝 Entrada manual de datos")
        st.write("Ingresa valores para cada feature y obtén una predicción instantánea")

        # Organizar inputs en columnas
        cols_per_row = 3
        inputs = {}

        for i in range(0, len(feature_names), cols_per_row):
            cols = st.columns(cols_per_row)
            for j, col in enumerate(cols):
                if i + j < len(feature_names):
                    feature = feature_names[i + j]
                    with col:
                        inputs[feature] = st.number_input(
                            f"📊 {feature}",
                            value=0.0,
                            format="%.6f",
                            key=f"manual_{feature}"
                        )

        input_df = pd.DataFrame([inputs])
        st.markdown('</div>', unsafe_allow_html=True)

        col1, col2, col3 = st.columns([1, 2, 1])
        with col2:
            if st.button("🚀 Predecir", use_container_width=True):
                with st.spinner('Calculando predicción...'):
                    try:
                        pred = pipe.predict(input_df)
                        probs = None
                        try:
                            if hasattr(pipe, 'predict_proba'):
                                probs = pipe.predict_proba(input_df)
                        except Exception:
                            probs = None

                        # Mostrar resultado con diseño atractivo
                        st.markdown('<div class="card">', unsafe_allow_html=True)
                        col1, col2 = st.columns(2)

                        with col1:
                            st.markdown(f"""
                            <div class="metric-container">
                                <div class="metric-label">Predicción</div>
                                <div class="metric-value">{pred[0]}</div>
                            </div>
                            """, unsafe_allow_html=True)

                        with col2:
                            if probs is not None and probs.shape[1] == 2:
                                st.markdown(f"""
                                <div class="metric-container">
                                    <div class="metric-label">Probabilidad</div>
                                    <div class="metric-value">{probs[0,1]:.2%}</div>
                                </div>
                                """, unsafe_allow_html=True)

                        st.markdown('</div>', unsafe_allow_html=True)

                        # Log
                        logpath = append_log(input_df, pred, probs)
                        st.info(f"📝 Registro guardado en: {logpath}")
                    except Exception as e:
                        st.error(f"❌ Error durante la predicción: {e}")
    else:
        st.warning("⚠️ Por favor, carga el modelo primero")

# TAB 2: Batch predictions
with tab2:
    st.markdown('<div class="card">', unsafe_allow_html=True)
    st.subheader("📤 Carga tu archivo CSV")
    st.write("Sube un archivo con múltiples registros para predicción masiva")

    uploaded = st.file_uploader(
        "Arrastra tu archivo o haz clic para seleccionar",
        type=['csv'],
        help="El archivo debe contener las mismas columnas que el dataset de entrenamiento"
    )

    if uploaded is not None and pipe is not None:
        try:
            df_new = pd.read_csv(uploaded)

            # Mostrar preview del archivo
            with st.expander("👁️ Vista previa del archivo", expanded=True):
                st.dataframe(df_new.head(10), use_container_width=True)
                st.write(f"📊 Dimensiones: {df_new.shape[0]} filas × {df_new.shape[1]} columnas")

            # Align columns
            expected = feature_names
            missing = [c for c in expected if c not in df_new.columns]

            if missing:
                st.error(f"❌ Faltan columnas: {', '.join(missing)}")
            else:
                df_aligned = df_new[expected].copy()

                col1, col2, col3 = st.columns([1, 2, 1])
                with col2:
                    if st.button("🎯 Predecir lote y guardar resultados", use_container_width=True):
                        with st.spinner('⏳ Procesando predicciones...'):
                            try:
                                preds = pipe.predict(df_aligned)
                                probs = None
                                try:
                                    if hasattr(pipe, 'predict_proba'):
                                        probs = pipe.predict_proba(df_aligned)
                                except Exception:
                                    probs = None

                                outdf = df_aligned.copy()
                                outdf['prediction'] = preds
                                if probs is not None and probs.shape[1] == 2:
                                    outdf['prob_pos'] = probs[:,1]

                                outdf.to_csv(PREDICTIONS_OUT, index=False)
                                append_log(df_aligned, preds, probs)

                                st.success(f"✅ Predicciones guardadas en: {PREDICTIONS_OUT}")

                                # Mostrar estadísticas
                                col1, col2, col3 = st.columns(3)
                                with col1:
                                    st.metric("Total predicciones", len(preds))
                                with col2:
                                    st.metric("Clase 0", (preds == 0).sum())
                                with col3:
                                    st.metric("Clase 1", (preds == 1).sum())

                                st.download_button(
                                    "⬇️ Descargar predicciones (CSV)",
                                    data=outdf.to_csv(index=False).encode('utf-8'),
                                    file_name='predicciones_resultado.csv',
                                    mime='text/csv',
                                    use_container_width=True
                                )
                            except Exception as e:
                                st.error(f"❌ Error durante predicción por lote: {e}")
        except Exception as e:
            st.error(f"❌ Error leyendo el CSV: {e}")

    st.markdown('</div>', unsafe_allow_html=True)

# TAB 3: Validation
with tab3:
    if pipe is None:
        st.info("⚠️ Carga el pipeline para usar el modo de validación")
    else:
        st.markdown('<div class="card">', unsafe_allow_html=True)
        st.subheader("🎯 Validación del modelo")
        st.write("Evalúa el rendimiento del modelo con datos etiquetados")

        uploaded_test = st.file_uploader(
            "Sube un CSV de test (debe contener 'target')",
            type=['csv'],
            key='test_upload',
            help="El archivo debe incluir la columna 'target' con las etiquetas reales"
        )

        use_uploaded = uploaded_test is not None

        if use_uploaded:
            df_test = pd.read_csv(uploaded_test)
        elif os.path.exists('/content/test_para_validacion.csv'):
            df_test = pd.read_csv('/content/test_para_validacion.csv')
            st.info("📂 Usando archivo de test por defecto")
        else:
            df_test = None

        if df_test is not None:
            with st.expander("👁️ Vista previa del dataset de test", expanded=False):
                st.dataframe(df_test.head(10), use_container_width=True)

            if 'target' not in df_test.columns:
                st.error("❌ El dataset de validación debe incluir la columna 'target'")
            else:
                missing = [c for c in feature_names if c not in df_test.columns]
                if missing:
                    st.error(f"❌ Faltan columnas en dataset de test: {', '.join(missing)}")
                else:
                    X_test = df_test[feature_names].copy()
                    y_test = df_test['target'].copy()

                    col1, col2, col3 = st.columns([1, 2, 1])
                    with col2:
                        if st.button("🧮 Correr validación y calcular métricas", use_container_width=True):
                            with st.spinner('📊 Calculando métricas...'):
                                try:
                                    preds = pipe.predict(X_test)
                                    probs = None
                                    try:
                                        if hasattr(pipe, 'predict_proba'):
                                            probs = pipe.predict_proba(X_test)
                                    except Exception:
                                        probs = None

                                    acc = accuracy_score(y_test, preds)
                                    prec = precision_score(y_test, preds, zero_division=0)
                                    rec = recall_score(y_test, preds, zero_division=0)
                                    f1s = f1_score(y_test, preds, zero_division=0)
                                    cm = confusion_matrix(y_test, preds)
                                    report = classification_report(y_test, preds, zero_division=0)

                                    st.markdown("---")
                                    st.subheader("📈 Resultados de Validación")

                                    # Métricas en tarjetas
                                    col1, col2, col3, col4 = st.columns(4)

                                    with col1:
                                        st.markdown(f"""
                                        <div class="metric-container">
                                            <div class="metric-label">Accuracy</div>
                                            <div class="metric-value">{acc:.2%}</div>
                                        </div>
                                        """, unsafe_allow_html=True)

                                    with col2:
                                        st.markdown(f"""
                                        <div class="metric-container">
                                            <div class="metric-label">Precision</div>
                                            <div class="metric-value">{prec:.2%}</div>
                                        </div>
                                        """, unsafe_allow_html=True)

                                    with col3:
                                        st.markdown(f"""
                                        <div class="metric-container">
                                            <div class="metric-label">Recall</div>
                                            <div class="metric-value">{rec:.2%}</div>
                                        </div>
                                        """, unsafe_allow_html=True)

                                    with col4:
                                        st.markdown(f"""
                                        <div class="metric-container">
                                            <div class="metric-label">F1-Score</div>
                                            <div class="metric-value">{f1s:.2%}</div>
                                        </div>
                                        """, unsafe_allow_html=True)

                                    st.markdown("---")

                                    # Matriz de confusión y reporte
                                    col1, col2 = st.columns(2)

                                    with col1:
                                        st.markdown("**🎯 Matriz de Confusión**")
                                        st.dataframe(
                                            pd.DataFrame(
                                                cm,
                                                columns=['Pred 0', 'Pred 1'],
                                                index=['Real 0', 'Real 1']
                                            ),
                                            use_container_width=True
                                        )

                                    with col2:
                                        st.markdown("**📋 Classification Report**")
                                        st.text(report)

                                    outdf = X_test.copy()
                                    outdf['target'] = y_test
                                    outdf['prediction'] = preds
                                    if probs is not None and probs.shape[1] == 2:
                                        outdf['prob_pos'] = probs[:,1]

                                    outdf.to_csv(VALIDATION_EVIDENCE, index=False)
                                    st.success(f"✅ Evidencia guardada en: {VALIDATION_EVIDENCE}")

                                    st.download_button(
                                        "⬇️ Descargar evidencia (CSV)",
                                        data=outdf.to_csv(index=False).encode('utf-8'),
                                        file_name='evidencia_validacion_predicciones.csv',
                                        mime='text/csv',
                                        use_container_width=True
                                    )
                                except Exception as e:
                                    st.error(f"❌ Error al ejecutar la validación: {e}")

        st.markdown('</div>', unsafe_allow_html=True)

# Footer mejorado
st.markdown("---")
with st.expander("📂 Archivos del sistema", expanded=False):
    file_status = []
    for f in [PREDICTIONS_OUT, VALIDATION_EVIDENCE, LOGFILE, MODEL_PATH, DF_TRAIN_PATH]:
        status = '✅ Existe' if os.path.exists(f) else '❌ No encontrado'
        file_status.append({'Archivo': f, 'Estado': status})

    st.dataframe(pd.DataFrame(file_status), use_container_width=True, hide_index=True)

st.markdown("""
<div style='text-align: center; padding: 2rem; color: #666;'>
    <p>💡 <strong>Tip:</strong> Para ejecutar la app en Colab: levanta Streamlit y ngrok/localtunnel</p>
    <p style='font-size: 0.9rem; margin-top: 1rem;'>Desarrollado con ❤️ usando Streamlit</p>
</div>
""", unsafe_allow_html=True)



Overwriting /content/app.py


In [ ]:
from pyngrok import ngrok
from pyngrok import ngrok, conf
import os, time, subprocess, signal

# Pega aquí tu token (reemplaza la cadena)
AUTHTOKEN = "35KjxESoOJSDnKzIglmLBtd2Due_7dmKy39hgYs5ccK1qjgar"

# Guardar el token en la configuración de pyngrok
!ngrok authtoken {AUTHTOKEN}


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import streamlit as st
import pandas as pd
import joblib
st.title("Demo Streamlit")
st.write("Coloca aquí tu app real.")
# Asegúrate de que 'modelo_final_pipeline.pkl' existe en /content si tu app lo usa.

# Levantar ngrok al puerto 8501 y mostrar URL pública
public_url = ngrok.connect(8501)
print("🌍 URL pública (ngrok):", public_url)

# Iniciar Streamlit en background
get_ipython().system_raw('streamlit run app.py --server.port 8501 &')

2025-11-11 15:47:21.788 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 15:47:21.789 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 15:47:21.790 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 15:47:21.791 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 15:47:21.792 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 15:47:21.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


🌍 URL pública (ngrok): NgrokTunnel: "https://overplentifully-costless-ardell.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
from pyngrok import ngrok

# Cerrar túneles previos por si acaso
ngrok.kill()

# Abrir un nuevo túnel HTTPS al puerto 8501
public_url = ngrok.connect(addr=8501, bind_tls=True)
print("🌍 URL pública (ngrok):", public_url)

🌍 URL pública (ngrok): NgrokTunnel: "https://overplentifully-costless-ardell.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
# 🔴 Detener ngrok (túnel público)
from pyngrok import ngrok
ngrok.kill()
# 🔴 Matar el proceso de Streamlit en el puerto 8501
!kill $(lsof -t -i:8501) || true

# # Verificar que ya no haya nada usando ese puerto
!lsof -i:8501 || echo "✅ Streamlit detenido correctamente"


COMMAND     PID USER   FD   TYPE  DEVICE SIZE/OFF NODE NAME
streamlit 42153 root    6u  IPv4 1121863      0t0  TCP *:8501 (LISTEN)
streamlit 42153 root    7u  IPv6 1121864      0t0  TCP *:8501 (LISTEN)
